<a href="https://colab.research.google.com/github/aplneto/IF702/blob/master/project_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

_file = 'drive/MyDrive/TRNcod.xls'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Separação dos dados em um arquivo para cada classe

import pandas as pd
import os

class_1 = 'drive/MyDrive/TRNcod-class1.csv'
class_2 = 'drive/MyDrive/TRNcod-class2.csv'

try:
  os.remove(class_1)
  os.remove(class_2)
except FileNotFoundError:
  print("Arquivos limpos")
else:
  print("Apagando arquivos")

count = [0, 0]

for index, chunk in enumerate(pd.read_table(_file, chunksize=100000)):
  print("\rProcessando %iº pedaço" % (index+1), end='')
  csv_c1 = chunk[chunk['IND_BOM_1_1'] == 1]
  csv_c2 = chunk[chunk['IND_BOM_1_2'] == 1]
  assert(len(chunk) == len(csv_c1) + len(csv_c2))
  count[0] += len(csv_c1)
  count[1] += len(csv_c2)
  csv_c1.to_csv(class_1, index=False, header=False, mode='a')
  csv_c2.to_csv(class_2, index=False, header=False, mode='a')
        
print("\rProcessamento do arquivo concluído")
headers = chunk.columns
print("Classe 1: %i\nClasse 2: %i" % tuple(count))

Arquivos limpos
Processamento do arquivo concluído
Classe 1: 255098
Classe 2: 134098


In [3]:
# Classe 1

import random
import progressbar

random.seed(4)

sizes = [count[0] - int(count[0] * 0.5)] + [int(count[0] * 0.25)] * 2
if sum(sizes) < count[0]: sizes[0] += 1

base_name = 'drive/MyDrive/if702/TRNcod-class'

files = [base_name+'1-train.csv', base_name+'1-valid.csv', base_name+'1-test.csv']

try:
  for filename in files:
    os.remove(filename)
except FileNotFoundError:
  pass

limits = dict(zip(files, sizes))
counts = dict(zip(files, [0, 0, 0]))
prob = dict(zip(files, [0.5, 0.25, 0.25]))
streams = dict(zip(files, [open(x, 'w') for x in files]))

bar = progressbar.ProgressBar(maxval=count[0], \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

t = 0

bar.start()
with open(class_1) as csv:
  line = csv.readline()
  while (line):
    bar.update(t)
    c = random.choices(files, weights = prob.values())[0]
    _f = streams[c]
    _f.write(line)
    counts[c] += 1
    if counts[c] >= limits[c]:
      files.remove(c)
      prob.pop(c)
      print("Arquivo %s Completo" % c)
      _f.close()
    line = csv.readline()
    t += 1
bar.finish()

print('\n')
print(counts)
print("Total: %i" % sum(counts.values()))

[========================================================================] 100%


Arquivo drive/MyDrive/if702/TRNcod-class1-train.csv Completo
Arquivo drive/MyDrive/if702/TRNcod-class1-test.csv Completo
Arquivo drive/MyDrive/if702/TRNcod-class1-valid.csv Completo


{'drive/MyDrive/if702/TRNcod-class1-train.csv': 127550, 'drive/MyDrive/if702/TRNcod-class1-valid.csv': 63774, 'drive/MyDrive/if702/TRNcod-class1-test.csv': 63774}
Total: 255098


In [4]:
class_1_sizes = []

for k in counts.keys():
  csv = pd.read_csv(k)
  print("%s: %i" % (k, len(csv)))
  class_1_sizes.append(len(csv))

drive/MyDrive/if702/TRNcod-class1-train.csv: 127549
drive/MyDrive/if702/TRNcod-class1-valid.csv: 63773
drive/MyDrive/if702/TRNcod-class1-test.csv: 63773


In [5]:
# Classe 2

sizes2 = [count[1] - int(count[1] * 0.5)] + [int(count[1] * 0.25)] * 2

files = [base_name+'2-train.csv', base_name+'2-valid.csv', base_name+'2-test.csv']

try:
  for filename in files:
    os.remove(filename)
except FileNotFoundError:
  pass

limits = dict(zip(files, sizes2))
counts = dict(zip(files, [0, 0, 0]))
prob = dict(zip(files, [0.5, 0.25, 0.25]))
streams = dict(zip(files, [open(x, 'w') for x in files]))

bar = progressbar.ProgressBar(maxval=count[1], \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

t = 0

bar.start()
with open(class_1) as csv:
  while (files):
    line = csv.readline()
    bar.update(t)
    c = random.choices(files, weights = prob.values())[0]
    _f = streams[c]
    _f.write(line)
    counts[c] += 1
    if counts[c] >= limits[c]:
      print(counts[c], limits[c])
      files.remove(c)
      prob.pop(c)
      print("Arquivo %s Completo" % c)
      _f.close
    t += 1

bar.finish()

print('\n')
print(counts)
print("Total: %i" % sum(counts.values()))

[========================================================================] 100%


33524 33524
Arquivo drive/MyDrive/if702/TRNcod-class2-valid.csv Completo
67049 67049
Arquivo drive/MyDrive/if702/TRNcod-class2-train.csv Completo
33524 33524
Arquivo drive/MyDrive/if702/TRNcod-class2-test.csv Completo


{'drive/MyDrive/if702/TRNcod-class2-train.csv': 67049, 'drive/MyDrive/if702/TRNcod-class2-valid.csv': 33524, 'drive/MyDrive/if702/TRNcod-class2-test.csv': 33524}
Total: 134097


In [6]:
class_2_sizes = []

for k in counts.keys():
  csv = pd.read_csv(k)
  print("%s: %i" % (k, len(csv)))
  class_2_sizes.append(len(csv))

drive/MyDrive/if702/TRNcod-class2-train.csv: 67025
drive/MyDrive/if702/TRNcod-class2-valid.csv: 33519
drive/MyDrive/if702/TRNcod-class2-test.csv: 33508


In [7]:
files = [base_name+'2-train.csv', base_name+'2-valid.csv', base_name+'2-test.csv']

train_file = open(files[0])
train_set = train_file.readlines()
train_file.close()
train_file = open(files[0], 'a')
lim = len(train_set)
while (lim < class_1_sizes[0]):
  line = random.choice(train_set)
  train_file.write(line)
  lim += 1
train_file.close()

In [8]:
csv = pd.read_csv(base_name+'2-train.csv')
len(csv)

127548

In [10]:
valid_file = open(files[1])
valid_set = valid_file.readlines()
valid_file.close()
valid_file = open(files[1], 'a')
lim = len(valid_set)
while (lim < class_1_sizes[1]):
  line = random.choice(valid_set)
  valid_file.write(line)
  lim+=1
valid_file.close()

In [11]:
csv = pd.read_csv(base_name+'2-valid.csv')
len(csv)

63772

In [12]:
# Embaralhando conjuntos

files = [base_name+'2-train.csv', base_name+'2-valid.csv', base_name+'2-test.csv'] + [base_name+'1-train.csv', base_name+'1-valid.csv', base_name+'1-test.csv']

for f in files:
  io = open(f)
  io_buff = io.readlines()
  io.close()
  random.shuffle(io_buff)
  io = open(f, 'w')
  while io_buff:
    io.write(io_buff.pop())

In [13]:
for k in files:
  csv = pd.read_csv(k)
  print("%s: %i" % (k, len(csv)))

drive/MyDrive/if702/TRNcod-class2-train.csv: 127548
drive/MyDrive/if702/TRNcod-class2-valid.csv: 63772
drive/MyDrive/if702/TRNcod-class2-test.csv: 33508
drive/MyDrive/if702/TRNcod-class1-train.csv: 127549
drive/MyDrive/if702/TRNcod-class1-valid.csv: 63773
drive/MyDrive/if702/TRNcod-class1-test.csv: 63760
